In [ ]:
# HTML output for this notebook can be produced using
# jupyter nbconvert --to html --no-input notebook_name.ipynb
# or
# jupyter nbconvert --to pdf --no-input notebook_name.ipynb
# edit title and authors in notebook metadata (e.g. jupyter lab / notebook tools / advanced)

This notebook contains the same functionality as `tidal_analysis_rt.ipynb`, but with all detail code extracted into the `../acoustic_tracking` package code folder. 

Analysis of acoustic tracking performance using range test data obtained in Mahone Bay near Halifax, NS, by OTN field experiments during March-April 2016.

Range test performance is determined with two methods:

  1. by calculating interval lenghts between adjacent detection events,
  1. by counting the number of detection events within some fixed time interval and normalizing against the expected number of detections.

Components of this notebook:

 * process tidal data for the time period considering high/low tide times and the observed heights
 * determine tidal phase timing
 * perform cosine interpolation of heights
 * correlate detection performance against tidal phase

**Data:** Beyond tidal data, environmental variables have been collected for 3 hour intervals. Water velocity is used from those variables to determine its potential effect on detection performance.

Summary plots are presented separately for each receiver / transmitter combination at the end of this notebook.

In [ ]:
try:
    from acoustic_tracking import *
except:
    from this_path import sys_path_append; sys_path_append("..")
    from acoustic_tracking import *

In [ ]:
# mpl.use('module://ipympl.backend_nbagg')
#%matplotlib widget
%matplotlib inline

rcParams['figure.figsize'] = 16, 8
rcParams['font.size'] = 14
rcParams["legend.framealpha"] = 0.6
rcParams['figure.dpi']= 300

## Detection data is merged with environmental variables

The current example merges HYCOM environmental data via **pre-processing that is not included in this notebook**.

Automated data fetching is one of the TODO items in this project.

In [ ]:
df_detections_merged = pd.read_csv(repo_file_path("Range_Test_VUE_Export_detections_use.csv", folder=""))

dt_format_str = "%m/%d/%y %H:%M:%S"
df_detections_merged["datetime"] = pd.to_datetime(df_detections_merged["Date and Time"],
                                                  format=dt_format_str)

In [ ]:
# TODO: source environmental data from pyERDAP or kadlu.fetch
# TODO verify that time zones in entire notebook are used correctly
df_detections_merged.head()

# Determine tidal heights via interpolation of tidal time tables

In addition to ocean and weather model data, historic tidal tables are available and used here to provide additional information about environmental cycles that could be factors of influence on the acoustic data.

In [ ]:
#dftt = pd.read_excel(file_path("Extracted_tidal_times_for_Halifax_2016.xlsx"), 0)
#print("Times are in UTC @ Halifax, Heights are in Centimetres @ Halifax")

In [ ]:
df_tidal_times = read_ods(repo_file_path("Extracted_tidal_times_for_Halifax_2016_2sheets.ods"), 1)

dflat = flatten_tidal_table(df_tidal_times, year=2016)
dflat.to_csv("output_tidal_times_for_Halifax_2016_flat.csv")

printmd("""
## Tidal data for Halifax
Linear interpolation
""")
dflat["height"].plot()
plt.ylabel("height (cm)")
plt.xlabel(None)
plt.grid()

In [ ]:
dfi = tidal_phase(dflat,
                  new_times = df_detections_merged.datetime)
#dfi

In [ ]:
fname = "output_tidal_times_for_Halifax_2016_5min.csv"
dfi.to_csv(fname)
printmd("Wrote data to `{}`".format(fname))

In [ ]:
#end_datetime = "2016-03-15 01:18"
with plt.rc_context({'figure.figsize': (16, 5), 'lines.linewidth': 2}):
    end_datetime = dfi.index.max()
    printmd("Display data until {}".format(end_datetime))
    dfi.loc[:end_datetime].height.plot()
    dfi.loc[:end_datetime].dheight_cm_per_hr.plot()
    (dfi.loc[:end_datetime].t*10).plot()
    plt.title("Tidal data for Halifax with Cosine interpolation")
    plt.legend(loc=1)
    plt.grid()

The variable $t$ above indicate tidal phase within each of high-to-low and low-to-high portion. Its range is in $[0,1]$, but has been magnified by a factor of $10$ in the plot to show more clearly in comparison to the other variables.

Below, a new variable $t2$ is introduced that ranges from $0$ to $2$, from high tide to the next high tide, with $1$ corresponding to low tide.

In [ ]:
dfmm = df_detections_merged.merge(dfi[["t2","height","dheight_cm_per_hr"]], left_on="datetime", right_index=True)

In [ ]:
dj = dfmm.set_index("datetime").sort_index()
del dj["Date"], dj["Time"], dj["Date and Time"]

# Group detection data by distinct "Receiver", "Transmitter" pairs
Later, each of these groups is analysed separately.
A name is produced for each pairing that reflects their configuration, such as power level, tag family, distance - as determined by parsing the metadata.

In [ ]:
djg = dj.groupby(["Receiver","Transmitter"])
groups = [djg.get_group(x) for x in djg.groups]
# list(map(len, groups))

# groups is a list of DataFrames that have the respective detections.

# Receiver / Transmitter metadata

In [ ]:
dfmeta = pd.read_csv(repo_file_path("range_test_raw.csv"))

File `range_test_raw.csv` does not have further metainfo merged in. This will be fixed with additional code, below.

In [ ]:
dfmeta.head()

## Ingest metadata, data dictionary, and deployment info

* Load data sheets
* Correct column names, convert integers, convert datetimes
* Merge Recv/Tag info with meta data, calculate geodesic distances


In [ ]:
metadata_file = repo_file_path("metadata-from-initial-range-test.xls")
mdb = make_Bunch("Container for metadata")
mdb.datadict, mdb.deploy = read_otn_metadata(metadata_file)

In [ ]:
display(Markdown("### Data dictionary"))
display_full_df(mdb.datadict)
display(Markdown("### Deployment info"))
display_full_df(mdb.deploy)

In [ ]:
deploy_lat_lon = mdb.deploy.groupby('STATION_NO')[['DEPLOY_LAT','DEPLOY_LONG']].nth(0)

mdb.station_dists_m = calc_station_dists_m(deploy_lat_lon)
display(Markdown("""### Station distances
* geodesic
* in meters
* ignoring depth difference

Distance between stations 2 and 3 does not occur in detections, since there are no receivers at these stations.
"""))
mdb.station_dists_m

### Processing of other metadata (custom format defined by SFU)

The first part of metadata stems from vendor CSV files, simply copying the relevant rows from different source CSVs into one table.
This contains High and Low power mode. This information is missing from the other metadata that is loaded later.

In [ ]:
tag_specs_df = pd.read_excel(repo_file_path("tag-specs-Mahone-Bay-range-test", "tag-summary-mahone-bay-range-test.xls"))
tag_specs = clean_vendor_tag_specs(tag_specs_df)
tag_specs

In [ ]:
display(Markdown("""### Merge tag ID Code with INS_SERIAL_NO to get metadata
The tags that have missing info here, turn out to be unimportant later, due to insufficient detection count.
"""))
iname = tag_specs.index.name
tag_specs_merged = tag_specs.reset_index().merge(mdb.deploy, 'left', left_on='ID Code', right_on='INS_SERIAL_NO').set_index(iname)
mdb.tag_specs = tag_specs_merged

display_full_df(tag_specs_merged)

In [ ]:
gsdf = get_all_group_info(groups, mdb)

In [ ]:
gsdf

In [ ]:
d_min, d_max = gsdf['dist_m'].max(), gsdf['dist_m'].min()
dist_th = np.mean((d_min, d_max))
def dist_str(dist):
    return dist_str_th(dist, dist_th)
gsdf['Receiver/Transmitter'] = gsdf['Receiver/Transmitter'] + "-" + gsdf['dist_m'].apply(dist_str)

In [ ]:
printmd("""# Summary of detections by Receiver/Transmitter pair
**R/T name format:**  
Receiver/Transmitter/Tag Family/Power(H,L)/Distance(Near,Far)

**Distances:**  
near = %.2f m  
far = %.2f m
""" % (d_min, d_max))

gsdf.sort_values(by="Receiver/Transmitter", ascending=False)

In [ ]:
# -- preparation for plotting section --

In [ ]:
# import imp; import acoustic_tracking; imp.reload(acoustic_tracking); from acoustic_tracking import *

In [ ]:
columns = ['salinity_bottom', 'water_temp_bottom', 'water_u_bottom', 'water_v_bottom',
            'salinity', 'water_temp', 'water_u', 'water_v']
colnames = dict(zip(columns, list(s.replace("_", " ") for s in columns)))
colnames.update({'water_vel':'water velocity',
                 'water_vel_bottom':'water velocity bottom'})
column = 'water_vel'

In [ ]:
printmd("# Plots of detection density and interval lengths <br/> against tidal phase (t2) and {}".format(colnames[column]))

In [ ]:
rcParams['figure.figsize'] = 16, 5
rcParams['font.size'] = 11
rcParams['figure.max_open_warning'] = 50

params = make_Bunch("Detection processing parameters")
params.t2bin_stepsize = 0.05
params.t2bins = np.arange(0, 2+1e-4, params.t2bin_stepsize)
params.base_interval_cutoff = 2**9
params.mean_ping_interval = 300 # sec
params.num_time_bins = 200
params.MIN_DETECTIONS = 100 # skip receiver/transmitter combinations that have less than this number of detections
params.rt_name_dist = lambda gr: rt_name(gr, mdb, dist_str)
params.mdr_params = make_Bunch("make_detection_rate parameters",
                               exp_interval_s=params.mean_ping_interval, num_time_bins=params.num_time_bins)
# technically, outputs are not parameters, might separate that later
params.out = make_Bunch("State and output of detection processing",
                        interval_all = np.zeros(len(params.t2bins)-1))
# column is defined above
column_name = (column, colnames[column])

skipmsg = False
# each group contains all detections for a particular receiver/transmitter combination
for gr in groups:
    if len(gr) < params.MIN_DETECTIONS:
        if not skipmsg:
            printmd("**Skipping receiver/transmitter combinations that have insufficient detections:**")
            skipmsg = True
        printmd("{}".format(params.rt_name_dist(gr)))
        continue
    
    # add interval length and water velocity calculations to dataframe
    tdfok, cutoff_t, tdf = clean_detections(gr, params, column_name=column_name)
    # TODO add tdfok to list for later pd.concat

    if True:
        plot_with_dr(tdfok, params, column_name=column_name)

    fig, axs = plt.subplots(nrows=1, ncols=3)
    fig.suptitle(params.rt_name_dist(gr))

    plot_tidal_phase(tdfok, ax=axs[0])

    plot_with_detection_count(params.out.tdfcount, params.out.tdfmean,
                              params, column_name=column_name,
                              ax=axs[0])    # interval lengths over date range
    if True:
        # comparison in single plot
        plot_with_detection_interval(tdfok, params, column_name=column_name, ax=axs[1])
        #ax = tdfmean.plot("t2","interval", alpha=1, ax=ax1, c="darkgrey", linewidth=2)
    elif False:
        # old plot type focussing on interval lengths rather than DR
        plot_with_detection_interval_and_rate(tdfok, params, column_name=column_name, ax=axs[1])
    else:
        # stacked plot for comparison of quantities
        plot_stack_with_dr(tdfok, params, column_name=column_name, mainax=axs[1])

    plot_per_detection_density(tdfok, params, column_name=column_name, ax=axs[2])
    plt.subplots_adjust(wspace=.3)

# Discussion

In the above visual summary, the **H-N** combinations, i.e. high-power, near distance, are the ones where water velocity shows the least effect on variations in detection rate (detection density). This confirms expectations and shows promise for the proposed study method. Next steps include:

- Continue to work with detection rate (DR) as calculated in a fixed grid of time windows
- Compare variations of DR with respect to other environmental variables
- Import other environmental variables automatically via data source APIs (ERDDAP, kadlu.fetch)
- Determine suitable numerical measure of factor importance in addition to visual analysis 


# Acknowledgements

The above analysis was performed using [data from OTN](http://members.devel.oceantrack.org/erddap/tabledap/otnunit_aat_detections.html) (provided by Jonathan Pye of OTN), in combination with HYCOM environmental data and tidal data provided by Casey Hilliard (Meridian/Dal), with a synthesized dataset prepared by Matthew Berkowitz (SFU), with project definition and guidance provided by Oliver Kirsebom (Dal) and Ines Hessler (Dal) as part of the [Meridian Network](https://meridian.cs.dal.ca).